<a href="https://colab.research.google.com/github/Pratikshya49/Concepts-and-Technologies-of-AI/blob/main/Worksheet6_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Logistic (Sigmoid) Function**

---



In [ ]:
import numpy as np

def logistic_function(x):
    """
    Computes the logistic (sigmoid) function.
    """
    return 1 / (1 + np.exp(-x))


In [ ]:
def test_logistic_function():
    x_scalar = 0
    assert round(logistic_function(x_scalar), 3) == 0.5

    x_pos = 2
    assert round(logistic_function(x_pos), 3) == round(1 / (1 + np.exp(-2)), 3)

    x_neg = -3
    assert round(logistic_function(x_neg), 3) == round(1 / (1 + np.exp(3)), 3)

    x_array = np.array([0, 2, -3])
    expected = np.array([0.5, 0.881, 0.047])
    assert np.all(np.round(logistic_function(x_array), 3) == expected)

    print("All tests passed!")

test_logistic_function()


All tests passed!


2. Log-Loss **Function** **bold text** bold text

In [ ]:
def log_loss(y_true, y_pred):
    import numpy as np

    y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
    loss = -(y_true * np.log(y_pred)) - ((1 - y_true) * np.log(1 - y_pred))
    return loss


In [ ]:
def test_log_loss():
    import numpy as np

    assert np.isclose(log_loss(1, 1), 0.0)
    assert np.isclose(log_loss(0, 0), 0.0)

    y_true, y_pred = 1, 0.8
    expected = -np.log(0.8)
    assert np.isclose(log_loss(y_true, y_pred), expected)

    y_true, y_pred = 0, 0.2
    expected = -np.log(0.8)
    assert np.isclose(log_loss(y_true, y_pred), expected)

    print("All tests passed!")

test_log_loss()


All tests passed!


**3. Cost Function (Average Log-Loss)**

In [ ]:
def cost_function(y_true, y_pred):
    assert len(y_true) == len(y_pred)

    n = len(y_true)
    loss_vec = log_loss(y_true, y_pred)
    cost = np.sum(loss_vec) / n
    return cost


In [ ]:
def test_cost_function():
    import numpy as np

    y_true = np.array([1, 0, 1])
    y_pred = np.array([0.9, 0.1, 0.8])

    expected_cost = (
        -np.log(0.9) - np.log(0.9) - np.log(0.8)
    ) / 3

    result = cost_function(y_true, y_pred)
    assert np.isclose(result, expected_cost, atol=1e-6)

    print("Test passed!")

test_cost_function()



Test passed!


**4.Vectorized Cost Function (Using Parameters)**

In [ ]:
def costfunction_logreg(X, y, w, b):
    n, d = X.shape

    z = np.dot(X, w) + b
    y_pred = logistic_function(z)
    cost = cost_function(y, y_pred)
    return cost


In [ ]:
X = np.array([[10, 20], [-10, 10]])
y = np.array([1, 0])
w = np.array([0.5, 1.5])
b = 1

print(costfunction_logreg(X, y, w, b))


5.500008350834906


**5. Gradient Computation**

In [ ]:
def compute_gradient(X, y, w, b):
    n, d = X.shape

    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    error = y_pred - y
    grad_w = (1 / n) * np.dot(X.T, error)
    grad_b = (1 / n) * np.sum(error)

    return grad_w, grad_b


In [18]:

X = np.array([[10, 20], [-10, 10]])
y = np.array([1, 0])
w = np.array([0.5, 1.5])
b = 1
# Assertion tests
try:
    grad_w, grad_b = compute_gradient(X, y, w, b)
    print("Gradients computed successfully.")
    print(f"grad_w: {grad_w}")
    print(f"grad_b: {grad_b}")
except AssertionError as e:
    print(f"Assertion error: {e}")

Gradients computed successfully.
grad_w: [-4.99991649  4.99991649]
grad_b: 0.4999916492890759


**6. Gradient Descent**

In [ ]:
def gradient_descent(X, y, w, b, alpha, n_iter, show_cost=False, show_params=True):
    cost_history = []
    params_history = []

    for i in range(n_iter):
        grad_w, grad_b = compute_gradient(X, y, w, b)

        w -= alpha * grad_w
        b -= alpha * grad_b

        cost = costfunction_logreg(X, y, w, b)

        cost_history.append(cost)
        params_history.append((w.copy(), b))

        if show_cost and (i % 100 == 0 or i == n_iter - 1):
            print(f"Iteration {i}: Cost = {cost:.6f}")
        if show_params and (i % 100 == 0 or i == n_iter - 1):
            print(f"Iteration {i}: w = {w}, b = {b:.6f}")

    return w, b, cost_history, params_history


In [ ]:
def test_gradient_descent():
    X = np.array([[0.1, 0.2], [-0.1, 0.1]])
    y = np.array([1, 0])
    w = np.zeros(2)
    b = 0.0

    w_out, b_out, cost_history, _ = gradient_descent(X, y, w, b, 0.1, 100)

    assert len(cost_history) == 100
    assert cost_history[-1] < cost_history[0]
    print("All tests passed!")

test_gradient_descent()

Iteration 0: w = [0.005  0.0025], b = 0.000000
Iteration 99: w = [0.49236201 0.24271295], b = -0.023120
All tests passed!


**7. Prediction Function**

In [ ]:
def prediction(X, w, b, threshold=0.5):
    z = np.dot(X, w) + b
    y_prob = logistic_function(z)
    y_pred = (y_prob >= threshold).astype(int)
    return y_pred

In [ ]:
def test_prediction():
    X = np.array([[0.5, 1.0], [1.5, -0.5], [-0.5, -1.0]])
    w = np.array([1.0, -1.0])
    b = 0.0

    expected = np.array([0, 1, 1])
    assert np.array_equal(prediction(X, w, b), expected)

    print("Test passed!")

test_prediction()

Test passed!


**8. Evaluation Metrics**

In [ ]:
def evaluate_classification(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    confusion_matrix = np.array([[TN, FP],
                                  [FN, TP]])

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return {
        "confusion_matrix": confusion_matrix,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }